In [2]:
import pandas as pd 
import numpy as np 
import psycopg2

In [3]:
conn = psycopg2.connect(
    dbname="prueba_sephora",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432")

In [4]:
cur = conn.cursor()

In [39]:
df_principal = pd.read_csv('productos_maquillaje.csv')
pd.set_option('display.max_columns', None)
df_principal

,categoria,subcategoria,marca,nombre,descripcion,precio,numero_valoraciones,valoracion,num_variaciones,fecha_extraccion,formato,responsabilidad,efecto_sombra,efecto_labios,efecto_mascara,tipo_brocha,formulacion,tipo_piel,cobertura,acabado,textura
0,Ojos,Sombras de ojos,Rem Beauty,Starlet,Sombra de ojos líquida,23.0,1,2.0,12,2025-04-15,NaN,NaN,"Mate, Irisado/Nacarado, Brillante",NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,NaN,Liquido
1,Rostro,Polvos bronceadores,Dior,Dior Forever Nude Bronze,Polvos bronceadores acabado radiante edición l...,65.0,0,0.0,2,2025-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,Ligera,Natural,NaN
2,Labios,Voluminizador labios,Dior Backstage,Dior Lip Glow Oil,Aceite para labios brillante nutritivo realzad...,48.0,434,4.2,3,2025-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,NaN,Aceite
3,Ojos,Paleta de sombras de ojos,Dior,Diorshow 5 Couleurs edición limitada,sombras de ojos de colores intensos,75.0,0,0.0,2,2025-04-15,NaN,NaN,Irisado/Nacarado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Uñas,Esmalte de uñas,Dior,Dior Vernis,Laca de uñas efecto gel color couture,37.0,0,0.0,2,2025-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Todo tipo de pieles,NaN,Purpurina,NaN


In [40]:
df_principal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   categoria            5 non-null      object 
 1   subcategoria         5 non-null      object 
 2   marca                5 non-null      object 
 3   nombre               5 non-null      object 
 4   descripcion          5 non-null      object 
 5   precio               5 non-null      float64
 6   numero_valoraciones  5 non-null      int64  
 7   valoracion           5 non-null      float64
 8   num_variaciones      5 non-null      int64  
 9   fecha_extraccion     5 non-null      object 
 10  formato              0 non-null      float64
 11  responsabilidad      0 non-null      float64
 12  efecto_sombra        2 non-null      object 
 13  efecto_labios        0 non-null      float64
 14  efecto_mascara       0 non-null      float64
 15  tipo_brocha          0 non-null      float64

In [ ]:
tabla_marcas = pd.DataFrame(df_principal["marca"].unique(), columns=["nombre_marca"])
tabla_marcas

,nombre_marca
0,Rem Beauty
1,Dior
2,Dior Backstage


In [8]:
data_to_insert = [[row["nombre_marca"]] for indice, row in tabla_marcas.iterrows()]

In [9]:
insert_query = """
        INSERT INTO marcas (nombre_marca)
        VALUES (%s)
"""

In [10]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:
tabla_categorias = pd.DataFrame(df_principal["categoria"].unique(), columns=["nombre_categoria"])
data_to_insert = [[row["nombre_categoria"]] for indice, row in tabla_categorias.iterrows()]
insert_query = """
        INSERT INTO categorias (nombre_categoria)
        VALUES (%s)
"""
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:
tabla_subcategorias = pd.DataFrame(df_principal["subcategoria"].unique(), columns=["nombre_subcategoria"])
data_to_insert = [[row["nombre_subcategoria"]] for indice, row in tabla_subcategorias.iterrows()]
insert_query = """
        INSERT INTO subcategorias (nombre_subcategoria)
        VALUES (%s)
"""
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [13]:
cur.execute("SELECT nombre_marca, id_marca FROM marcas")
marcas_dict = dict(cur.fetchall()) 
marcas_dict

{'Rem Beauty': 1, 'Dior': 2, 'Dior Backstage': 3}

In [14]:
cur.execute("SELECT nombre_categoria, id_categoria FROM categorias")
categorias_dict = dict(cur.fetchall()) 
categorias_dict

{'Ojos': 1, 'Rostro': 2, 'Labios': 3, 'Uñas': 4}

In [15]:
cur.execute("SELECT nombre_subcategoria, id_subcategoria FROM subcategorias")
subcategorias_dict = dict(cur.fetchall()) 
subcategorias_dict

{'Sombras de ojos': 1,
 'Polvos bronceadores': 2,
 'Voluminizador labios': 3,
 'Paleta de sombras de ojos': 4,
 'Esmalte de uñas': 5}

In [ ]:
data_to_insert = []
df_productos = df_principal[["nombre", "descripcion", "marca", "categoria", "subcategoria"]]
for _, row in df_productos.iterrows(): 
    nombre = row["nombre"]
    descripcion = row["descripcion"]
    id_marca = marcas_dict.get(row["marca"]) if row["marca"] in marcas_dict else None
    id_categoria = categorias_dict.get(row["categoria"]) if row["categoria"] in categorias_dict else None
    id_subcategoria = subcategorias_dict.get(row["subcategoria"]) if row["subcategoria"] in subcategorias_dict else None
    data_to_insert.append([nombre, descripcion, id_marca, id_categoria, id_subcategoria]) 

tabla_productos = pd.DataFrame(data_to_insert, columns=["nombre", "descripcion", "id_marca", "id_categoria", "id_subcategoria"])
tabla_productos

,nombre,descripcion,id_marca,id_categoria,id_subcategoria
0,Starlet,Sombra de ojos líquida,1,1,1
1,Dior Forever Nude Bronze,Polvos bronceadores acabado radiante edición l...,2,2,2
2,Dior Lip Glow Oil,Aceite para labios brillante nutritivo realzad...,3,3,3
3,Diorshow 5 Couleurs edición limitada,sombras de ojos de colores intensos,2,1,4
4,Dior Vernis,Laca de uñas efecto gel color couture,2,4,5


In [27]:
insert_query = """
        INSERT INTO productos (nombre, descripcion, id_marca, id_categoria, id_subcategoria)
        VALUES (%s, %s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [28]:
cur.execute("SELECT nombre, id_producto FROM productos")
productos_dict = dict(cur.fetchall()) 
productos_dict

{'Starlet': 1,
 'Dior Forever Nude Bronze': 2,
 'Dior Lip Glow Oil': 3,
 'Diorshow 5 Couleurs edición limitada': 4,
 'Dior Vernis': 5}

In [30]:
data_to_insert = []
df_historico = df_principal[["nombre", "fecha_extraccion", "precio", "numero_valoraciones", "valoracion", "num_variaciones"]]
for _, row in df_historico.iterrows(): 
    id_producto = productos_dict.get(row["nombre"]) if row["nombre"] in productos_dict else None
    fecha_extraccion = row["fecha_extraccion"]
    precio = row["precio"]
    numero_valoraciones = row["numero_valoraciones"]
    valoracion = row["valoracion"]
    numero_variaciones = row["num_variaciones"]
    data_to_insert.append([id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones]) 

tabla_historico = pd.DataFrame(data_to_insert, columns=["id_producto", "fecha_extraccion", "precio", "numero_valoraciones", "valoracion", "numero_variaciones"])
tabla_historico

,id_producto,fecha_extraccion,precio,numero_valoraciones,valoracion,numero_variaciones
0,1,2025-04-15,23.0,1,2.0,12
1,2,2025-04-15,65.0,0,0.0,2
2,3,2025-04-15,48.0,434,4.2,3
3,4,2025-04-15,75.0,0,0.0,2
4,5,2025-04-15,37.0,0,0.0,2


In [31]:
insert_query = """
        INSERT INTO historico (id_producto, fecha_extraccion, precio, numero_valoraciones, valoracion, numero_variaciones)
        VALUES (%s, %s, %s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

TABLAS FILTROS

In [33]:
df_efectos_sombra = df_principal["efecto_sombra"].dropna()
efectos_sombra_list = df_efectos_sombra.apply(lambda x: [ef.strip() for ef in x.split(",")])
efectos_sombra_unicos = set([efecto for sublist in efectos_sombra_list for efecto in sublist])

tabla_efectos_sombra = pd.DataFrame(efectos_sombra_unicos, columns=["nombre_efecto"])

data_to_insert = [[row["nombre_efecto"]] for _, row in tabla_efectos_sombra.iterrows()]

tabla_efectos_sombra

,nombre_efecto
0,Brillante
1,Irisado/Nacarado
2,Mate


In [37]:
cur.execute("SELECT nombre_efecto, id_efecto_sombra FROM efectos_sombra")
efectos_sombra_dict = dict(cur.fetchall()) 
efectos_sombra_dict

{'Brillante': 1, 'Irisado/Nacarado': 2, 'Mate': 3}

In [38]:
data_to_insert = []
for _, row in df_principal.iterrows():
    if pd.isna(row["efecto_sombra"]):
        continue

    id_producto = productos_dict.get(row["nombre"])

    efectos_sombra = [ef.strip() for ef in row["efecto_sombra"].split(",")]
    
    for efecto in efectos_sombra:
        id_efecto_sombra = efectos_sombra_dict.get(efecto)
        if id_producto and id_efecto_sombra:
            data_to_insert.append((id_producto, id_efecto_sombra))

insert_query = """
    INSERT INTO producto_efecto_sombra (id_producto, id_efecto_sombra)
    VALUES (%s, %s)
"""  

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [34]:
insert_query = """
    INSERT INTO efectos_sombra (nombre_efecto)
    VALUES (%s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [35]:
df_texturas = df_principal["textura"].dropna()
texturas_list = df_texturas.apply(lambda x: [tx.strip() for tx in x.split(",")])
texturas_unicos = set([textura for sublist in texturas_list for textura in sublist])

tabla_texturas = pd.DataFrame(texturas_unicos, columns=["nombre_textura"])

data_to_insert = [[row["nombre_textura"]] for _, row in tabla_texturas.iterrows()]

tabla_texturas

,nombre_textura
0,Liquido
1,Aceite


In [36]:
insert_query = """
    INSERT INTO texturas (nombre_textura)
    VALUES (%s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()